**CSE 5522 Lab3**

**Author: Xuecheng Liu**

**Part 1**

**load the data frame**

In [0]:
import pandas as pd
import numpy as np
import random

## if you want to test tbe program with different data, please go to 
## https://github.com/Xuecheng-Liu/CSE-5522-data-file to edit the corresponding csv file
obsUrl = "https://raw.githubusercontent.com/Xuecheng-Liu/CSE-5522-data-file/master/observationProbs.csv"
transUrl = "https://raw.githubusercontent.com/Xuecheng-Liu/CSE-5522-data-file/master/transitionProbs.csv"
testUrl = "https://raw.githubusercontent.com/Xuecheng-Liu/CSE-5522-data-file/master/testData.csv"

## load the data frame
obs = pd.read_csv(obsUrl)
trans = pd.read_csv(transUrl)
test = pd.read_csv(testUrl)

#display(obs)
#display(trans)
#display(test)


**construct the probability matrix from data frame and initialize the first row in magic matric**

In [0]:
# matrix A: Transition matrix
transition = np.array(trans.drop(trans.columns[0],axis=1),dtype='float')
#display(transition)
# matrix B: Emission matrix
emission = np.array(obs.drop(obs.columns[0],axis=1),dtype = 'float')
#display(emission)

sequence = np.array(test.drop(test.columns[0],axis=1),dtype='int')
seq_1 = sequence[0]

#display(seq_1)

## pi: prior probabilities for first day to be cold and hot
priors_start_with_C = float(transition[0][2])
priors_start_with_H = float(transition[1][2])

y1 = seq_1[0]
B_cold = emission[y1-1][0] # eat n ice cream corresponds to n-1 row in emission matrix
B_hot = emission[y1-1][1]

## initialize the first row of the "magic matrix"
M_1_cold = priors_start_with_C * B_cold
M_1_hot = priors_start_with_H * B_hot
#display(M_1_cold)
#display(M_1_hot)

**go through each sequence of observations and make predictions**

In [62]:
pred_state=[]
for row in range(10):
  seq = sequence[row]
  seq_1 = []
  for i in seq:
    if i !=0:
      seq_1.append(i)
  last_state_c = []
  last_state_h =[]
  
  y1 = seq_1[0]
  B_cold = emission[y1-1][0]
  B_hot = emission[y1-1][1]
  M_1_cold = priors_start_with_C * B_cold
  M_1_hot = priors_start_with_H * B_hot

  for i in range(1,len(seq_1)):
    ob = seq_1[i] 
    c_to_c = trans_state[0][0] * emission[ob-1][0] * M_1_cold
    h_to_c = trans_state[1][0] * emission[ob-1][0] * M_1_hot

    c_to_h = trans_state[0][1] * emission[ob-1][1] * M_1_cold
    h_to_h = trans_state[1][1] * emission[ob-1][1] * M_1_hot

    C_i_c = np.argmax([trans_state[0][0] * M_1_cold,trans_state[0][1] * M_1_hot])
    C_i_h = np.argmax([trans_state[1][0]* M_1_cold,trans_state[1][1] * M_1_hot])

    last_state_c.append(C_i_c)
    last_state_h.append(C_i_h)
    
    # 0 for cold and 1 for hot
    if c_to_c > h_to_c:
      M_1_cold = c_to_c
    else:
      M_1_cold = h_to_c
    
    if c_to_h > h_to_h:
      M_1_hot = c_to_h
    else:
      M_1_hot = h_to_h

  Xt = np.argmax([M_1_cold,M_1_hot])
  result = []
  result.append(Xt)
  i = len(seq_1)-1
  
  while i > 0:
    if Xt == 0:
      result.append(last_state_c[i-1])
      Xt = last_state_c[i-1]
    else:
      result.append(last_state_h[i-1])
      Xt = last_state_h[i-1]
    i = i-1

  # convert the result to the weather sequence
  print("Sequence "+ str(row + 1) +": ")
  print(['cold' if x == 0 else 'hot' for x in result])
  pred_state.append(result)

Sequence 1: 
['hot', 'hot', 'hot', 'hot', 'hot']
Sequence 2: 
['hot', 'hot', 'hot', 'hot']
Sequence 3: 
['cold', 'cold', 'cold', 'cold', 'cold']
Sequence 4: 
['cold', 'cold', 'cold']
Sequence 5: 
['cold', 'cold', 'cold', 'cold', 'cold']
Sequence 6: 
['cold', 'cold', 'cold', 'cold']
Sequence 7: 
['hot', 'hot', 'hot']
Sequence 8: 
['cold', 'cold', 'hot', 'hot']
Sequence 9: 
['hot', 'hot', 'hot', 'hot', 'cold']
Sequence 10: 
['cold', 'cold', 'cold']


**Part 2**

**normalize the transition probability**

In [64]:
norm_tran = np.array(trans.drop(trans.columns[0],axis=1),dtype="float")
n = np.delete(norm_tran,2,0)
a = n[0][0]+n[1][0]
n[0][0] = n[0][0]/a
n[1][0] = 1 - n[0][0]
b = n[0][1]+n[1][1]
n[0][1]=n[0][1]/b
n[1][1]=1-n[0][1]
new_trans = n
display(new_trans)

array([[0.92473118, 0.07526882, 0.5       ],
       [0.07526882, 0.92473118, 0.5       ]])

In [70]:
for a in range(10):
  seq = sequence[a]
  seq_1 = []
  for i in seq:
    if i !=0:
      seq_1.append(i)
  
  iteration = random.randint(10,100001)
  
  w = 1
  w_dict=[]
  weight = []
  for i in range(iteration):
    weather_seq =[]
    rd = random.uniform(0,1)
    current_state = 0 
    if rd > 0.5:
      current_state = 1
    event=seq_1[0]
    w = w * emission[event-1][current_state]
    weather_seq.append(current_state)

    ## iterate from 2 to last state
    for j in range(1,len(seq_1)):
      rd = random.uniform(0,1)
      next_state = 1
      if rd <= new_trans[current_state][0]:
        next_state = 0
      event = seq_1[j] 
      w = w * emission[event-1][next_state]
      current_state = next_state
      weather_seq.append(current_state)
    
    
    weight.append(w)
    w_dict.append(weather_seq)
  
  p = 0
  total = 0
  num_obs_seq = 0
  num = 0
  for i in w_dict:
    if np.array_equal(pred_state[num_obs_seq],i):
      p += weight[num]
    else:
      total += weight[num]
    num+=1
  num_obs_seq+=1
  
  total+=p

  print("Sequence "+str(a+1)+" with probility: "+str(abs(w-p/total)))
  print(['cold' if x == 0 else 'hot' for x in result])




Sequence 1 with probility: 1.1646733020579101e-20
['cold', 'cold', 'cold']
Sequence 2 with probility: 0.0
['cold', 'cold', 'cold']
Sequence 3 with probility: 1.369034516560235e-26
['cold', 'cold', 'cold']
Sequence 4 with probility: 0.0
['cold', 'cold', 'cold']
Sequence 5 with probility: 1.98984296e-12
['cold', 'cold', 'cold']
Sequence 6 with probility: 0.0
['cold', 'cold', 'cold']
Sequence 7 with probility: 0.0
['cold', 'cold', 'cold']
Sequence 8 with probility: 0.0
['cold', 'cold', 'cold']
Sequence 9 with probility: 9.77290744609022e-09
['cold', 'cold', 'cold']
Sequence 10 with probility: 0.0
['cold', 'cold', 'cold']


The probability of sampling each sequence is different since we are generating random iterations every time